# Converting volumn mixing ratio to column density

In running RRTMG column model, if `IATM = 1` in the INPUT_RRTM, then molecule concentrations can be given in a variety of units listed in Table II in the instructions.  Choosing `IATM = 1` also means that RRTATM, a "ray-tracing program" originally from LBLRTM, will be used.  It is in this program that, if the user-specified atmosphere profile pressures and altitudes are not appropriately accurate, errors such as 

     BOUNDARIES ARE OUTSIDE THE RANGE OF THE ATMOSPHERE
     BOUNDARY =       2.85 ATMOSPHERE =      2.85
     RESET BOUNDARY GT THAN ATMOSPHERE
     BOUNDARIES OUTSIDE OF ATMOS
     
might be raised.  

If IATM = 0, the molecule concentrations need to be as column densities, with units of `molecules/cm**2`.  It is these column densities that are input into the subroutine `setcoef_sw()`, which is the earliest common subroutine between the the global version and the column version of RRTMG.  In the global model, before `setcoef_sw` is called, another subroutine, `inatm_sw` is called.  In this subroutine, amongst other things, the input molecules densities in volumne mixing ratio are converted to column densities in `molecules/cm**2`.  `inatm_sw` is not included in the column model, which calls its own `readprof()` before `setcoef_sw()`.  `readprof()` simply reads whatever values are in INPUT_RRTM and pass them to `setcoef_sw()`.  

Therefore, if we want to use `IATM = 0` and we only have molecule densities as mixing ratios to begin with, we need to use the appropriate part of `inatm_sw` to convert mixing ratios to column densities before writing them to INPUT_RRTM.

Instead of compiling `inatm_sw()` with f2py3, the relevant expressions will be extracted and tranlated into Python, because `inatm_sw()` also does many other things and require other input variables, which, in turn, need to be looked for. 

In [1]:
%matplotlib inline

import os
import imp

import numpy as np
import pandas as pd
import xarray as xr

import climatools.io.cesm as climaio
import climatools.units as climaunits

In [2]:
# constants

AMD = 28.9660
AMW = 18.0160
AVOGAD = 6.02214199

GRAV = 9.8066

In [ ]:
def column_density_dryair(h2o=None, pdp=None):
    '''
    Parameters
    ----------
    h2o: H2o volume mixing ratio
    pdp: layer pressure difference [hPa, mb]
    coldry: dry air column density [molecules/cm**2]
    '''
    dims = ('time', 'lat', 'lon', '')
    amm = (1 - h2o) * AMD + h2o * AMW
    coldry = pdp * 1e3 * AVOGAD / (1e2 * GRAV * AMM * (1 + h2o))
    return coldry    


def ds_column_density_dryair(ds=None):
    '''
    Compute dry air column density and add it to dataset
    '''
    
    coldry = column_density_dryair(h2o=ds['h2ovmr'],
                                   pdp=ds['dpressure'])
    
    ds.update({'COLDRY': (('time', 'lat'))})
    
    